In [79]:
import pandas as pd  
import sqlite3

In [80]:
df = pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [81]:
conn = sqlite3.connect('churn')
df.to_sql('Churn',conn, if_exists =  'replace', index = False)
print("CSV has been connected to Database")

CSV has been connected to Database


In [82]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Churn LIMIT 10;")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 15634602, 'Hargrave', 619, 'France', 'Female', 42.0, 2, 0.0, 1, 1.0, 1.0, 101348.88, 1)
(2, 15647311, 'Hill', 608, 'Spain', 'Female', 41.0, 1, 83807.86, 1, 0.0, 1.0, 112542.58, 0)
(3, 15619304, 'Onio', 502, 'France', 'Female', 42.0, 8, 159660.8, 3, 1.0, 0.0, 113931.57, 1)
(4, 15701354, 'Boni', 699, 'France', 'Female', 39.0, 1, 0.0, 2, 0.0, 0.0, 93826.63, 0)
(5, 15737888, 'Mitchell', 850, 'Spain', 'Female', 43.0, 2, 125510.82, 1, None, 1.0, 79084.1, 0)
(6, 15574012, 'Chu', 645, 'Spain', 'Male', 44.0, 8, 113755.78, 2, 1.0, 0.0, 149756.71, 1)
(7, 15592531, 'Bartlett', 822, None, 'Male', 50.0, 7, 0.0, 2, 1.0, 1.0, 10062.8, 0)
(8, 15656148, 'Obinna', 376, 'Germany', 'Female', 29.0, 4, 115046.74, 4, 1.0, 0.0, 119346.88, 1)
(9, 15792365, 'He', 501, 'France', 'Male', 44.0, 4, 142051.07, 2, 0.0, None, 74940.5, 0)
(10, 15592389, 'H?', 684, 'France', 'Male', None, 2, 134603.88, 1, 1.0, 1.0, 71725.73, 0)


In [83]:
cursor.execute("""SELECT
Gender,
COUNT(*) AS Total,
SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) AS Churned,
ROUND(100.0* SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END)/ COUNT(*),2) AS ChurnRate
FROM Churn
GROUP BY Gender
ORDER BY ChurnRate DESC;
""")
results = cursor.fetchall()

for row in results:
    gender = row[0]
    total =  row[1]
    churned = row[2]
    churnrate = row[3]
    
print(f"""GENDER CHURN ANALYSIS
Gender = {gender}
Total COUNT = {total}
Total Churned = {churned}
ChurnRate = {churnrate}%
""")  

GENDER CHURN ANALYSIS
Gender = Male
Total COUNT = 5458
Total Churned = 899
ChurnRate = 16.47%



In [84]:
cursor.execute("""SELECT 
    Gender,
    COUNT(*) AS AgeChurnCount
FROM Churn  
WHERE Exited = 1 AND Age > 50
GROUP BY Gender;
""")

results = cursor.fetchall()

for row in results:
    gender = row[0]
    count = row[1]

print(f"The Gender Churn is : {gender} and The Age Churn is : {count} ")


The Gender Churn is : Male and The Age Churn is : 244 


In [ ]:
cursor.execute("""SELECT
Tenure,
COUNT(*) AS Total,
SUM(CASE WHEN Exited = 1 Then 1 ELSE 0 END) AS Churned,
ROUND(100.0 * SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Churn
GROUP BY Tenure
ORDER BY Tenure;
""")

results = cursor.fetchall()
for row in results:
    Tenure

In [85]:
cursor.execute("SELECT AVG(age) FROM Churn")
age = cursor.fetchone()[0]
print(f"The Average Of the Customers is {age:.2f}")

The Average Of the Customers is 38.92


In [86]:
cursor.execute("""SELECT AVG(EstimatedSalary) FROM Churn
""")

avg_salary = cursor.fetchone()[0]
print(f"The Average Salary of the Customers is {avg_salary:.2f}$")

The Average Salary of the Customers is 100083.33$


In [87]:
cursor.execute(""" SELECT Age , COUNT(*) AS ChurnCount
FROM Churn
WHERE Exited = 1
GROUP BY Age
ORDER BY ChurnCount DESC
LIMIT 5;
""")

results = cursor.fetchall()

for row in results:
    age = row[0]
    count = row[1]
    print(f"""TOP 5 AGES WITH HIGHEST CHURN
    Age = {age}
    Count = {count}
    """)


TOP 5 AGES WITH HIGHEST CHURN
    Age = 46.0
    Count = 91
    
TOP 5 AGES WITH HIGHEST CHURN
    Age = 40.0
    Count = 89
    
TOP 5 AGES WITH HIGHEST CHURN
    Age = 43.0
    Count = 88
    
TOP 5 AGES WITH HIGHEST CHURN
    Age = 45.0
    Count = 87
    
TOP 5 AGES WITH HIGHEST CHURN
    Age = 48.0
    Count = 80
    


In [88]:
cursor.execute("""SELECT Customerid , Geography , Gender , Age , EstimatedSalary
FROM Churn
WHERE Exited = 1 
ORDER BY EstimatedSalary DESC
LIMIT 5;
""")

result = cursor.fetchall()

for row in result:
    ID_s = row[0]
    Geography = row[1]
    Gender = row[2]
    Age = row[3]
    Salary = row[4]
    print(f""" TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = {ID_s}
    Geography = {Geography}
    Gender = {Gender}
    Age = {Age}
    Estimated Salary = {Salary}$
    """)


 TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = 15815656
    Geography = Germany
    Gender = Female
    Age = 39.0
    Estimated Salary = 199808.1$
    
 TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = 15661670
    Geography = Germany
    Gender = Female
    Age = 31.0
    Estimated Salary = 199725.39$
    
 TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = 15672152
    Geography = Germany
    Gender = Male
    Age = 37.0
    Estimated Salary = 199693.84$
    
 TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = 15661903
    Geography = France
    Gender = Female
    Age = 43.0
    Estimated Salary = 199378.58$
    
 TOP 5 CUSTOMERS WITH HIGHEST SALARY WHO CHURNED
    Customer ID = 15755262
    Geography = Spain
    Gender = Female
    Age = 41.0
    Estimated Salary = 199304.74$
    


In [89]:
cursor.execute("""SELECT AVG(CreditScore) FROM Churn
""")

avg_cred_score = cursor.fetchone()[0]
print(f"The Average Credit Score by Customer is {avg_cred_score:.2f}")

The Average Credit Score by Customer is 650.56


In [90]:
cursor.execute("""SELECT Geography FROM Churn 
WHERE Exited = 0
GROUP BY Geography
ORDER BY AVG(Estimatedsalary) DESC
LIMIT 1
;
""")

high_exits = cursor.fetchone()[0]
print(f"The Country with Highest Retention is {high_exits}")

The Country with Highest Retention is Germany


In [91]:
cursor.execute("""SELECT Geography FROM Churn
WHERE Exited = 1
GROUP BY Geography
ORDER BY AVG(Estimatedsalary) DESC
LIMIT 1
""")
low_exits = cursor.fetchone()[0]
print(f"The Country with Lowest Retention is {low_exits}")


The Country with Lowest Retention is Spain
